# 상품 추천 시스템
### Amazon SageMaker로 개인화 추천 시스템 구축 및 배포 실습

코드 구동에 필요한 라이브러리 불러오기

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import time

import boto3
import sagemaker
import sagemaker.amazon.common as smac

from scipy.sparse import csr_matrix, hstack, save_npz, load_npz
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 50)  # default is 50

In [ ]:
region = boto3.Session().region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client("s3", region_name=region)

sagemaker_boto_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)
sagemaker_role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()
print(f"using bucket{bucket} in region {region} \n")

## 데이터 불러오기
[Amazon Customer Review Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)
<br>
해당 실습에서는 모바일기기 카테고리의 리뷰데이터를 이용해서 모델을 생성. 전체 데이터 목록은 해당 [버킷](https://s3.console.aws.amazon.com/s3/buckets/amazon-reviews-pds?region=us-east-1&tab=objects)에서 확인 가능

In [ ]:
# 파일 다운로드
!wget -c https://amazon-reviews-pds.s3.amazonaws.com/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz
!wget -c https://amazon-reviews-pds.s3.amazonaws.com/tsv/amazon_reviews_us_Software_v1_00.tsv.gz

In [ ]:
# 파일이 정상적으로 다운로드 됐는지 확인
!ls

In [ ]:
# 다운받은 CSV 파일을 Pandas의 DataFrame으로 불러오기
df_mobile = pd.read_csv("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz", compression='gzip',
                    sep="\t", usecols=range(0, 15))
df_sw = pd.read_csv("amazon_reviews_us_Software_v1_00.tsv.gz", compression='gzip',
                    sep="\t", usecols=range(0, 15))
df = pd.concat([df_mobile, df_sw], axis=0)

In [ ]:
# 데이터 갯수 확인 밑 데이터 미리보기
print("Total records:", df.shape[0], "\n")
print("Sample records:\n")
df.sample(5)

In [ ]:
# DataFrame 정보 확인
df.info()

## 데이터 전처리

### 중복된 데이터 확인
구매자가 동일한 물품을 여러번 구매하고 다수의 리뷰를 남겼을 경우 혹은 물품을 한번 구매했지만 다수의 리뷰를 남길 경우

In [ ]:
duplicates = df.groupby(["customer_id","product_id", "product_title", "product_category"]).nunique()["review_id"]
duplicates = duplicates.loc[duplicates > 1].reset_index().rename(columns={'review_id': 'unique_reviews'})
print("Number of records with duplicates:", duplicates.shape[0], "\n")
duplicates

In [ ]:
# 한명의 구매자가 동일한 품목에 대해서 다수의 리뷰를 남긴 경우
df.loc[(df["customer_id"]==53086549) & (df["product_id"]=="B0000E6NKA")]

In [ ]:
# 레코드들을 구매자번호, 상품번호, 리뷰작성날짜로 정렬
df.sort_values(by=['customer_id', 'product_id', 'review_date'], inplace=True)
df.loc[(df["customer_id"]==53086549) & (df["product_id"]=="B0000E6NKA")]

In [ ]:
# 한명의 구매자가 동일한 품목에 대해서 다수의 리뷰를 남긴 경우, 가장 최근 리뷰만 남기고 삭제
df.drop_duplicates(['customer_id', 'product_id'], keep='last', inplace=True)

print("Dataset after dropping duplicates, number of rows and columns:", df.shape, "\n")

In [ ]:
# 위에서 확인한 중복 리뷰가 삭제됬는지 확인
df.loc[(df["customer_id"]==53086549) & (df["product_id"]=="B0000E6NKA")]

### 결측값(Missing values) 확인
특정 필드에 값이 들어가지 않을 경우

In [ ]:
df.isna().sum()

In [ ]:
# 별점값이 누락된 레코드들을 삭제
df.dropna(axis=0, subset=["star_rating"], inplace=True) 

In [ ]:
# 별점값이 누락된 레코드가 있는지 확인 
df['star_rating'].isna().sum()

### 유의미한 데이터 확인
실제 구매후 작성된 리뷰인지 확인

In [ ]:
# 구매 확인이 되지 않은 리뷰 확인
df[df["verified_purchase"]=="N"]

In [ ]:
# 실구매 확인되지 않은 리뷰 삭제 
df = df[df["verified_purchase"]=="Y"]

### 모델 생성 불필요한 필드 삭제

In [ ]:
# 구매자번호, 상품번호, 상품명, 상품분류, 별점만 데이터셋에 포함
columns = ["customer_id", "product_id", "product_title", "product_category", "star_rating"]
df = df[columns]

In [ ]:
# DataFrame에 있는 행(Row)들을 무작위로 재정렬
df = df.sample(frac=1, random_state=73)

In [ ]:
# 전처리가 완료된 데이터셋에 결측값이 존재하는지 확인
df.isna().sum()

In [ ]:
# 전처리가 완료된 DataFrame 확인
df.info()

## 탐색적 데이터 분석(Exploratory Data Analysis)

### 상품 카테고리 분포도

In [ ]:
# 상품 카테고리별 리뷰 갯수
df['product_category'].value_counts()

In [ ]:
# 상품 카테코리별 리뷰 갯수를 Bar 차트로 표현 
plt.style.use('fivethirtyeight')
df['product_category'].value_counts().sort_index().plot.bar(rot=0,
                                                            title="Product Category"
                                                           )
plt.title("Product Category", y=1.08)
plt.show()

### 상품 카테고리별 구매자 및 상품 분포도

In [ ]:
# 총 구매자 숫자와 상품 갯수 확인
customers = df.groupby(["product_category"]).nunique()["customer_id"].reset_index().rename(columns={'customer_id': 'unique_customers'})
products = df.groupby(["product_category"]).nunique()["product_id"].reset_index().rename(columns={'product_id': 'unique_products'})
customers_products = pd.merge(customers, products, on="product_category")
customers_products

In [ ]:
# 총 구매자 숫자와 상품 갯수를 Bar 차트로 표현
y1 = customers_products["unique_customers"].max()
y2 = customers_products["unique_products"].max()
y_max = y1 if y1>y2 else y2
y_max = np.ceil(y_max/10000)*10000

customers_products.plot(x="product_category", kind="bar", stacked=False,
           ylabel="", ylim=(0, y_max),
           xlabel="", rot=0)

plt.legend(loc="center", bbox_to_anchor=(1.1, 0.8))
plt.title("Unique Customers and Products \nby Product Category", y=1.08)
plt.show()

### 별점 분포도

In [ ]:
# 별점 점수별 갯수
df['star_rating'].value_counts()

In [ ]:
# 별점 점수별 갯수를 Bar 차트로 표현
plt.style.use('fivethirtyeight')
df['star_rating'].value_counts().sort_index().plot.bar(rot=0)
plt.title("Star Rating", y=1.08)
plt.show()

## 희소 행렬(Sparse Matrix) 생성

### 범주형 데이터(Categorical Data)에 One-Hot Encoding 수행

In [ ]:
ohe = OneHotEncoder(handle_unknown = "ignore")
ohe_cols = ["customer_id", "product_id", "product_category"]
ohe_features = ohe.fit_transform(df[ohe_cols])
ohe_features

In [ ]:
# 각 카테고리별 고유값 갯수 확인 
df['product_category'].nunique() + df['customer_id'].nunique() + df['product_id'].nunique()

In [ ]:
ohe_feature_names = ohe.get_feature_names()
df_ohe = pd.DataFrame(data = ohe_features.toarray(), index=range(len(df)), columns=ohe_feature_names)
df_ohe

In [ ]:
# 전체 데이터셋에서 10%만 샘플링
df_frac = df.sample(frac=0.1)
df_frac.info()

In [ ]:
# 각 카테고리별 고유값 갯수 재확인 
df_frac['product_category'].nunique() + df_frac['customer_id'].nunique() + df_frac['product_id'].nunique()

In [ ]:
# One-Hot Encoding 재수행
ohe = OneHotEncoder(handle_unknown = "ignore")
ohe_cols = ["customer_id", "product_id", "product_category"]
ohe_features = ohe.fit_transform(df_frac[ohe_cols])
ohe_features

In [ ]:
ohe_feature_names = ohe.get_feature_names()
df_ohe = pd.DataFrame(data = ohe_features.toarray(), index=range(len(df_frac)), columns=ohe_feature_names)
df_ohe

### 문자열 데이터에 TF-IDF Encoding 수행

In [ ]:
# 2개 미만의 문서에 포함된 문자는 제외
vectorizer = TfidfVectorizer(min_df=2)  
vectorizer.fit(df_frac["product_title"].unique())
tfidf_features = vectorizer.transform(df_frac["product_title"])
tfidf_features

In [ ]:
tfidf_feature_names = vectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_features.toarray(), index=range(len(df_frac)), columns=tfidf_feature_names)
df_tfidfvect

### 희소 행렬 생성

In [ ]:
# 위에서 인코딩한 데이터셋 통합
X = hstack([ohe_features, tfidf_features], format="csr", dtype="float32")
X

In [ ]:
y = df_frac["star_rating"].values.astype("float32")
y

In [ ]:
# 희소성 확인
total = X.shape[0] * X.shape[1]
non_zero = X.nnz
sparsity = (total - non_zero) / total

print("Total elements:", total)
print("Non-zero elements:", non_zero)
print("Sparsity:", round(sparsity*100, 4), "%")

## 데이터를 용도에 맞게 분리(Training, Test) 

In [ ]:
# 전체 데이터셋의 80%를 학습용으로 20%를 테스트용으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73)           

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("")
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

## 데이터를 RecordIO 파일로 변환

In [ ]:
# RecordIO 파일 생성
train_key = "fm_train.recordio"
test_key = "fm_test.recordio"

with open(train_key, 'wb') as f:
        smac.write_spmatrix_to_sparse_tensor (f, X_train, y_train)
        
with open(test_key, 'wb') as f:
        smac.write_spmatrix_to_sparse_tensor (f, X_test, y_test)

In [ ]:
# 생성한 RecordIO 파일을 S3로 저장
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

boto3.resource('s3').Bucket(bucket).upload_file(train_key, train_key)
boto3.resource('s3').Bucket(bucket).upload_file(test_key, test_key)

print("SageMaker version:", sagemaker.__version__)
print("Region:", region)
print("Bucket:", bucket)


## 모델 학습

In [ ]:
# Sagemaker에 부여된 IAM 역할 불러오기
role = sagemaker.get_execution_role()
role

In [ ]:
# 학습에 사용할 컨테이너 불러오기
container = sagemaker.image_uris.retrieve("factorization-machines", region=region)
container

In [ ]:
# Training Job 설정
fm = sagemaker.estimator.Estimator(    
    container,
    role,
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    output_path = f"s3://{bucket}",
    sagemaker_session = sess
)

In [ ]:
# Hyperparameter 설정
fm.set_hyperparameters(
    feature_dim = X.shape[1],
    num_factors = 64,  
    predictor_type = "regressor",
    epochs = 50,      
    mini_batch_size = 1000,  
)

fm.hyperparameters()

In [ ]:
# 학습 시작
fm.fit({'train': f"s3://{bucket}/{train_key}", 'test': f"s3://{bucket}/{test_key}"})

In [ ]:
# 완료된 Training Job 정보 확인
job_name = fm.latest_training_job.job_name

sagemaker_boto_client = boto3.Session(region_name=region).client("sagemaker")
training_job_info = sagemaker_boto_client.describe_training_job(TrainingJobName = job_name)
training_job_info

## 모델 배포

In [ ]:
# Request 및 Response가 JSON으로 처리되도록 희소행렬을 Json으로 변환할 Serializer 생성 
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
import json

class fm_json_serializer(JSONSerializer):
    def serialize(self, data):
        js = {"instances": []}
        for row in data:
            js["instances"].append({"features": row.tolist()})
        return json.dumps(js)

In [ ]:
# 모델 배포
predictor = fm.deploy(initial_instance_count = 1,
                             instance_type = "ml.m5.xlarge",
                             endpoint_name = job_name,
                             serializer = fm_json_serializer(),
                             deserializer = JSONDeserializer(),
                            )

## 모델 추론

### 단골 고객 확인

In [ ]:
### 리뷰를 많이 작성한 고객순으로 고객 목록 확인
df_frac.groupby("customer_id").count()["product_id"].sort_values(ascending=False).head(20)

In [ ]:
### 리뷰를 가장 많이 작성한 고객의 리뷰 확인
df_frac[df_frac["customer_id"] == 20602687]  

### 인기상품 목록

In [ ]:
# 가장 많은 고객이 리뷰한 상품순으로 상품 확인
trending = df_frac.copy()
trending = (trending.groupby(["product_id", "product_title", "product_category"])
            .nunique()["customer_id"]
            .sort_values(ascending=False)
            .reset_index()            
           )            
trending = trending.rename(columns={'customer_id': 'unique_customers'})
trending

### 각 카테고리별 Top 5 인기상품을 추천상품군에 포함

In [ ]:
trending_sw = trending[trending["product_category"]=="Software"].head(5)
trending_me = trending[trending["product_category"]=="Mobile_Electronics"].head(5)
trending_pool = pd.concat([trending_sw, trending_me], axis=0)
trending_pool

### 추론에 사용할 Input 데이터 생성

In [ ]:
# 고객번호 추가
trending_pool["customer_id"] = 20602687
trending_pool

In [ ]:
# One-Hot Encoding 수행
ohe = OneHotEncoder(handle_unknown = "ignore")
ohe_cols = ["customer_id", "product_id", "product_category"]
ohe.fit(df_frac[ohe_cols])
ohe_features = ohe.transform(trending_pool[ohe_cols])
ohe_features

In [ ]:
# TF-IDF Encoding 수행
vectorizer = TfidfVectorizer(min_df=2)
vectorizer.fit(df_frac["product_title"].unique())
tfidf_features = vectorizer.transform(trending_pool["product_title"])
tfidf_features

In [ ]:
# 위에서 인코딩한 Inpout 데이터 통합
X_trending = hstack([ohe_features, tfidf_features], format="csr", dtype="float32")
X_trending

In [ ]:
# Input 데이터 확인
X_trending.toarray()

### 선택된 고객에 추천상품군에 있는 상품들에 부여할 별점 예측

In [ ]:
result = predictor.predict(X_trending.toarray())
result

In [ ]:
predictions = [i["score"] for i in result["predictions"]]
predictions

In [ ]:
index_array = np.array(predictions).argsort()
index_array

In [ ]:
products = ohe.inverse_transform(ohe_features)[:, 1]
products

### 선택된 고객이 좋아할만한 3개지 상품 추천

In [ ]:
top_3_recommended = np.take_along_axis(products, index_array, axis=0)[: -4 : -1]
top_3_recommended

In [ ]:
# Array를 Dataframe으로 변환
df_3 = pd.DataFrame(top_3_recommended, columns=["product_id"])
df_3

In [ ]:
# 상품 상세정보 추가
df_recommend = pd.merge(df_3, trending_pool, on="product_id")
columns = ["product_id", "product_title", "product_category"]
df_recommend = df_recommend[columns]
df_recommend

## 추론 서버 삭제

In [ ]:
# predictor.delete_endpoint()